# Test du Modèle en Deux Étapes pour la Classification des Prunes Africaines

Ce notebook vous permet de tester le modèle en deux étapes pour la classification des prunes africaines. Le modèle fonctionne en deux étapes :
1. **Détection** : Détermine si l'image contient une prune ou non
2. **Classification** : Si une prune est détectée, détermine son état (bonne qualité, non mûre, tachetée, fissurée, meurtrie ou pourrie)

## Étapes :
1. Configuration de l'environnement
2. Connexion à Google Drive
3. Chargement des modèles entraînés
4. Téléchargement et test d'images
5. Visualisation des résultats

## 1. Configuration de l'environnement

Commençons par installer les dépendances nécessaires et configurer l'environnement.

In [ ]:
# Installer les dépendances
!pip install torch torchvision numpy matplotlib pillow

# Importer les bibliothèques nécessaires
import os
import sys
import json
import numpy as np
import torch
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
from google.colab import drive, files
import io
import zipfile

## 2. Connexion à Google Drive

Connectons-nous à Google Drive pour accéder aux modèles entraînés.

In [ ]:
# Monter Google Drive
drive.mount('/content/drive')

# Définir le chemin vers le dossier contenant les modèles entraînés
# Modifiez ce chemin si vos modèles sont stockés ailleurs
models_dir = '/content/drive/MyDrive/african_plums_models'

# Créer le dossier s'il n'existe pas
os.makedirs(models_dir, exist_ok=True)

# Vérifier le contenu du dossier
!ls -la {models_dir}

## 3. Téléchargement et configuration du projet

Téléchargez le projet corrigé et configurez-le pour l'utilisation.

In [ ]:
# Créer les répertoires nécessaires
!mkdir -p /content/plum_classifier_corrected/data
!mkdir -p /content/plum_classifier_corrected/models
!mkdir -p /content/plum_classifier_corrected/scripts

# Définir les classes pour la détection et la classification
detection_class_names = ['plum', 'non_plum']
classification_class_names = ['unaffected', 'unripe', 'spotted', 'cracked', 'bruised', 'rotten']

# Créer un fichier model_info.json temporaire
model_info = {
    "detection_class_names": detection_class_names,
    "classification_class_names": classification_class_names,
    "img_size": 224,
    "detection_threshold": 0.7
}

with open(os.path.join(models_dir, 'two_stage_model_info.json'), 'w') as f:
    json.dump(model_info, f, indent=4)

## 4. Définition des classes et fonctions nécessaires

Définissons les classes et fonctions nécessaires pour le modèle en deux étapes.

In [ ]:
# Définir les transformations pour les images
def get_val_transforms(img_size=224):
    return transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

# Fonction pour prétraiter une image
def preprocess_single_image(image_path, transform=None):
    if transform is None:
        transform = get_val_transforms()
    
    if isinstance(image_path, str):
        image = Image.open(image_path).convert('RGB')
    else:
        image = image_path.convert('RGB')
        
    return transform(image).unsqueeze(0)  # Ajouter une dimension de batch

# Définir l'architecture du modèle
class PlumClassifier(torch.nn.Module):
    """Modèle de classification des prunes."""
    def __init__(self, num_classes=6, base_model='resnet18', pretrained=True, dropout_rate=0.5):
        super(PlumClassifier, self).__init__()
        
        self.base_model_name = base_model
        self.num_classes = num_classes
        self.dropout_rate = dropout_rate
        
        # Sélectionner le modèle de base
        if base_model == 'resnet18':
            from torchvision.models import resnet18
            self.base_model = resnet18(pretrained=pretrained)
            num_features = self.base_model.fc.in_features
            self.base_model.fc = torch.nn.Identity()  # Retirer la dernière couche
            
        elif base_model == 'resnet50':
            from torchvision.models import resnet50
            self.base_model = resnet50(pretrained=pretrained)
            num_features = self.base_model.fc.in_features
            self.base_model.fc = torch.nn.Identity()
            
        elif base_model == 'mobilenet_v2':
            from torchvision.models import mobilenet_v2
            self.base_model = mobilenet_v2(pretrained=pretrained)
            num_features = self.base_model.classifier[1].in_features
            self.base_model.classifier = torch.nn.Identity()
        
        # Classifier personnalisé avec dropout pour la régularisation
        self.classifier = torch.nn.Sequential(
            torch.nn.Dropout(dropout_rate),
            torch.nn.Linear(num_features, 512),
            torch.nn.ReLU(),
            torch.nn.Dropout(dropout_rate/2),
            torch.nn.Linear(512, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, num_classes)
        )
        
    def forward(self, x):
        features = self.base_model(x)
        return self.classifier(features)
    
    def get_model_info(self):
        return {
            "base_model": self.base_model_name,
            "num_classes": self.num_classes,
            "dropout_rate": self.dropout_rate
        }

class LightweightPlumClassifier(torch.nn.Module):
    """Version légère du classificateur de prunes."""
    def __init__(self, num_classes=6, pretrained=True):
        super(LightweightPlumClassifier, self).__init__()
        
        self.num_classes = num_classes
        
        # Utiliser MobileNetV2 qui est plus léger
        from torchvision.models import mobilenet_v2
        self.base_model = mobilenet_v2(pretrained=pretrained)
        num_features = self.base_model.classifier[1].in_features
        self.base_model.classifier = torch.nn.Sequential(
            torch.nn.Dropout(0.2),
            torch.nn.Linear(num_features, num_classes)
        )
        
    def forward(self, x):
        return self.base_model(x)
    
    def get_model_info(self):
        return {
            "base_model": "mobilenet_v2_lightweight",
            "num_classes": self.num_classes,
            "dropout_rate": 0.2
        }

class TwoStageModel:
    """Modèle en deux étapes pour la détection et la classification des prunes."""
    def __init__(self, detection_model, classification_model, detection_threshold=0.7):
        self.detection_model = detection_model
        self.classification_model = classification_model
        self.detection_threshold = detection_threshold
        
    def predict(self, image_tensor, device):
        # Déplacer l'image sur le device
        image_tensor = image_tensor.to(device)
        
        # Étape 1 : Détection de prune
        self.detection_model.eval()
        with torch.no_grad():
            detection_outputs = self.detection_model(image_tensor)
            detection_probs = torch.nn.functional.softmax(detection_outputs, dim=1)[0]
            
            # Classe 0 = prune, Classe 1 = non-prune
            is_plum = detection_probs[0] > self.detection_threshold
            
        # Si ce n'est pas une prune, retourner le résultat
        if not is_plum:
            return False, "non_plum", detection_probs.cpu().numpy()
        
        # Étape 2 : Classification de l'état de la prune
        self.classification_model.eval()
        with torch.no_grad():
            classification_outputs = self.classification_model(image_tensor)
            classification_probs = torch.nn.functional.softmax(classification_outputs, dim=1)[0]
            _, predicted_idx = torch.max(classification_outputs, 1)
            
        return True, predicted_idx.item(), classification_probs.cpu().numpy()
    
    def get_model_info(self):
        detection_info = self.detection_model.get_model_info()
        classification_info = self.classification_model.get_model_info()
        
        return {
            "detection_model": detection_info,
            "classification_model": classification_info,
            "detection_threshold": self.detection_threshold
        }

def get_model(model_name='standard', num_classes=6, base_model='resnet18', pretrained=True):
    """Factory function pour créer un modèle."""
    if model_name == 'standard':
        return PlumClassifier(num_classes=num_classes, base_model=base_model, pretrained=pretrained)
    elif model_name == 'lightweight':
        return LightweightPlumClassifier(num_classes=num_classes, pretrained=pretrained)
    else:
        raise ValueError(f"Modèle '{model_name}' non supporté")

def get_two_stage_model(detection_model_name='lightweight', classification_model_name='standard',
                       detection_base_model='mobilenet_v2', classification_base_model='resnet18',
                       num_detection_classes=2, num_classification_classes=6,
                       pretrained=True, detection_threshold=0.7):
    """Factory function pour créer un modèle en deux étapes."""
    detection_model = get_model(
        model_name=detection_model_name,
        num_classes=num_detection_classes,
        base_model=detection_base_model,
        pretrained=pretrained
    )
    
    classification_model = get_model(
        model_name=classification_model_name,
        num_classes=num_classification_classes,
        base_model=classification_base_model,
        pretrained=pretrained
    )
    
    return TwoStageModel(detection_model, classification_model, detection_threshold)

## 5. Chargement du modèle en deux étapes

Chargeons le modèle en deux étapes à partir des fichiers sauvegardés.

In [ ]:
def load_two_stage_model(detection_model_path, classification_model_path, model_info_path, device):
    """Charge le modèle en deux étapes à partir des fichiers sauvegardés."""
    # Charger les informations du modèle
    with open(model_info_path, 'r') as f:
        model_info = json.load(f)
    
    # Extraire les informations nécessaires
    detection_class_names = model_info['detection_class_names']
    classification_class_names = model_info['classification_class_names']
    detection_threshold = model_info['detection_threshold']
    
    # Créer le modèle en deux étapes
    two_stage_model = get_two_stage_model(
        detection_model_name='lightweight',
        classification_model_name='standard',
        detection_base_model='mobilenet_v2',
        classification_base_model='resnet18',
        num_detection_classes=len(detection_class_names),
        num_classification_classes=len(classification_class_names),
        pretrained=False,  # Nous utilisons nos propres poids entraînés
        detection_threshold=detection_threshold
    )
    
    # Charger les poids entraînés si les fichiers existent
    if os.path.exists(detection_model_path):
        two_stage_model.detection_model.load_state_dict(torch.load(detection_model_path, map_location=device))
        print(f"Modèle de détection chargé depuis: {detection_model_path}")
    else:
        print(f"Fichier de modèle de détection non trouvé: {detection_model_path}")
        print("Utilisation d'un modèle pré-entraîné pour la détection.")
    
    if os.path.exists(classification_model_path):
        two_stage_model.classification_model.load_state_dict(torch.load(classification_model_path, map_location=device))
        print(f"Modèle de classification chargé depuis: {classification_model_path}")
    else:
        print(f"Fichier de modèle de classification non trouvé: {classification_model_path}")
        print("Utilisation d'un modèle pré-entraîné pour la classification.")
    
    # Mettre les modèles en mode évaluation
    two_stage_model.detection_model.eval()
    two_stage_model.classification_model.eval()
    
    return two_stage_model, model_info

# Déterminer le device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation de: {device}")

# Chemins vers les fichiers de modèle
detection_model_path = os.path.join(models_dir, 'detection_best_acc.pth')
classification_model_path = os.path.join(models_dir, 'classification_best_acc.pth')
model_info_path = os.path.join(models_dir, 'two_stage_model_info.json')

# Charger le modèle en deux étapes
print("Chargement du modèle en deux étapes...")
model, model_info = load_two_stage_model(
    detection_model_path,
    classification_model_path,
    model_info_path,
    device
)

## 6. Fonction de prédiction et de visualisation

Définissons les fonctions pour prédire et visualiser les résultats.

In [ ]:
def predict_image(model, image_path, model_info, device, transform=None):
    """Prédit si l'image contient une prune et, si oui, son état."""
    # Prétraiter l'image
    if transform is None:
        img_size = model_info.get('img_size', 224)
        transform = get_val_transforms(img_size)
    
    image_tensor = preprocess_single_image(image_path, transform)
    
    # Prédiction
    is_plum, predicted_idx, probs = model.predict(image_tensor, device)
    
    if is_plum:
        # C'est une prune, retourner la classe prédite
        predicted_class = model_info['classification_class_names'][predicted_idx]
        return True, predicted_class, None, probs
    else:
        # Ce n'est pas une prune
        return False, "non_plum", probs, None

def visualize_prediction(image_path, is_plum, predicted_class, detection_probs=None, classification_probs=None, model_info=None):
    """Visualise l'image avec sa prédiction."""
    # Charger l'image
    if isinstance(image_path, str):
        image = Image.open(image_path).convert('RGB')
    else:
        image = image_path.convert('RGB')
    
    # Créer la figure
    plt.figure(figsize=(12, 6))
    
    # Afficher l'image
    plt.subplot(1, 2, 1)
    plt.imshow(image)
    if is_plum:
        plt.title(f"Prédiction : Prune - {predicted_class}")
    else:
        plt.title("Prédiction : Pas une prune")
    plt.axis('off')
    
    # Afficher les probabilités
    plt.subplot(1, 2, 2)
    
    if is_plum and classification_probs is not None and model_info is not None:
        # Afficher les probabilités de classification
        class_names = model_info['classification_class_names']
        y_pos = np.arange(len(class_names))
        plt.barh(y_pos, classification_probs, align='center')
        plt.yticks(y_pos, class_names)
        plt.xlabel('Probabilité')
        plt.title('Probabilités par classe')
    elif not is_plum and detection_probs is not None and model_info is not None:
        # Afficher les probabilités de détection
        class_names = model_info['detection_class_names']
        y_pos = np.arange(len(class_names))
        plt.barh(y_pos, detection_probs, align='center')
        plt.yticks(y_pos, class_names)
        plt.xlabel('Probabilité')
        plt.title('Probabilités de détection')
    
    plt.tight_layout()
    plt.show()
    
    # Afficher les probabilités en pourcentage
    print("\nProbabilités détaillées :")
    if is_plum and classification_probs is not None and model_info is not None:
        for i, (cls, prob) in enumerate(zip(model_info['classification_class_names'], classification_probs)):
            print(f"{cls}: {prob*100:.2f}%")
    elif not is_plum and detection_probs is not None and model_info is not None:
        for i, (cls, prob) in enumerate(zip(model_info['detection_class_names'], detection_probs)):
            print(f"{cls}: {prob*100:.2f}%")

## 7. Téléchargement et test d'images

Maintenant, vous pouvez télécharger vos propres images et les tester avec le modèle en deux étapes.

In [ ]:
# Fonction pour télécharger et tester une image
def upload_and_predict():
    print("Veuillez télécharger une image :")
    uploaded = files.upload()
    
    for filename in uploaded.keys():
        # Charger l'image
        image = Image.open(io.BytesIO(uploaded[filename]))
        
        # Faire la prédiction
        is_plum, predicted_class, detection_probs, classification_probs = predict_image(
            model,
            image,
            model_info,
            device
        )
        
        # Afficher les résultats
        if is_plum:
            print(f"L'image contient une prune de type: {predicted_class}")
        else:
            print("L'image ne contient pas de prune.")
        
        # Visualiser la prédiction
        visualize_prediction(
            image,
            is_plum,
            predicted_class,
            detection_probs,
            classification_probs,
            model_info
        )

# Exécuter la fonction pour télécharger et tester une image
upload_and_predict()

## 8. Test avec plusieurs images

Vous pouvez également tester plusieurs images à la fois en les téléchargeant depuis votre Google Drive.

In [ ]:
# Fonction pour tester des images depuis un dossier
def test_images_from_folder(folder_path):
    # Vérifier si le dossier existe
    if not os.path.exists(folder_path):
        print(f"Le dossier {folder_path} n'existe pas.")
        return
    
    # Récupérer toutes les images du dossier
    image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif')
    image_files = [f for f in os.listdir(folder_path) 
                  if os.path.isfile(os.path.join(folder_path, f)) and 
                  f.lower().endswith(image_extensions)]
    
    if not image_files:
        print(f"Aucune image trouvée dans {folder_path}.")
        return
    
    print(f"Trouvé {len(image_files)} images à tester.")
    
    # Tester chaque image
    results = []
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        is_plum, predicted_class, detection_probs, classification_probs = predict_image(
            model,
            image_path,
            model_info,
            device
        )
        results.append((image_file, is_plum, predicted_class))
    
    # Afficher les résultats
    num_cols = min(3, len(results))
    num_rows = (len(results) + num_cols - 1) // num_cols
    
    plt.figure(figsize=(15, 5 * num_rows))
    
    for i, (image_file, is_plum, predicted_class) in enumerate(results):
        plt.subplot(num_rows, num_cols, i + 1)
        img = Image.open(os.path.join(folder_path, image_file))
        plt.imshow(img)
        if is_plum:
            plt.title(f"{image_file}\nPrédiction : Prune - {predicted_class}")
        else:
            plt.title(f"{image_file}\nPrédiction : Pas une prune")
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Afficher un tableau récapitulatif
    print("\nRécapitulatif des prédictions :")
    print("-" * 60)
    print(f"{'Image':<20} {'Type':<15} {'Classe prédite':<20}")
    print("-" * 60)
    
    for image_file, is_plum, predicted_class in results:
        type_str = "Prune" if is_plum else "Pas une prune"
        print(f"{image_file[:18]:<20} {type_str:<15} {predicted_class:<20}")
    
    return results

# Spécifiez le chemin vers un dossier contenant des images dans votre Google Drive
test_folder = input("Entrez le chemin vers le dossier contenant vos images de test : ")

# Tester les images du dossier
if test_folder:
    test_images_from_folder(test_folder)
else:
    print("Aucun chemin spécifié. Vous pouvez télécharger des images individuelles avec la cellule précédente.")

## 9. Télécharger d'autres images pour les tester

Vous pouvez continuer à tester d'autres images en exécutant à nouveau la cellule ci-dessous.

In [ ]:
# Exécutez cette cellule pour tester d'autres images
upload_and_predict()

## 10. Interface simple pour tester plusieurs images

Voici une interface simple pour tester plusieurs images à la fois.

In [ ]:
# Interface simple pour tester plusieurs images
from ipywidgets import widgets
from IPython.display import display, clear_output

def on_upload_button_clicked(b):
    clear_output()
    display(upload_button)
    upload_and_predict()

upload_button = widgets.Button(
    description='Télécharger et tester une image',
    button_style='info',
    tooltip='Cliquez pour télécharger une image à tester'
)

upload_button.on_click(on_upload_button_clicked)
display(upload_button)

## 11. Résumé et prochaines étapes

Félicitations ! Vous avez maintenant un notebook fonctionnel pour tester votre modèle en deux étapes pour la classification des prunes africaines.

### Ce que vous avez accompli :
- Configuré un modèle en deux étapes qui détecte d'abord si l'image contient une prune, puis classifie son état
- Testé des images individuelles téléchargées
- Testé des lots d'images depuis un dossier
- Visualisé les résultats de prédiction avec les probabilités

### Prochaines étapes possibles :
- Entraîner le modèle avec vos propres données
- Intégrer ce modèle dans une application Django (voir le guide dans le dépôt)
- Optimiser davantage le modèle en ajustant les hyperparamètres
- Créer une interface utilisateur plus élaborée pour le test
- Préparer une présentation pour le hackathon

N'hésitez pas à adapter ce notebook à vos besoins spécifiques pour le hackathon JCIA 2025 !